# Let a user choose an artist and song based on which images are to be selected and GAN images generated

In [1]:
import pickle
import pandas as pd
import numpy as np
import ast
import torch

In [2]:
with open('embeddings_lyrics.pkl', 'rb') as file:
    embeddings_lyrics = pickle.load(file)

In [3]:
with open('embeddings_artemis.pkl', 'rb') as file:
    embeddings_artemis = pickle.load(file)

In [4]:
path = "dataframe_tfidf_emotion.xlsx"
dataframe_songs = pd.read_excel(path)


### Select a song from the dataframe

##### Feel free to filter by artist:

In [89]:
artist = input("Enter artist name: ")

Enter artist name:  The Beatles


In [90]:
dataframe_songs[dataframe_songs['artist'] == artist]

,track,artist,clean_text,top_words,pred,label,top_words_sentence
1096,I Am the Walrus [take 16][Version],The Beatles,together see run like pigs gun see fly im cryi...,"['crying', 'goo', 'sitting', 'im', 'teeshirt',...",5,sadness,crying goo sitting im teeshirt gjoob see eggma...
1881,No Reply,The Beatles,happened came door reply said wasnt saw peep w...,"['saw', 'nearly', 'died', 'forgi', 'cause', 'd...",6,excitement,saw nearly died forgi cause door hand peep lig...
2024,Keep Your Hands off My Baby [#],The Beatles,weve friends oh long let share whats mine mess...,"['hands', 'keep', 'bayeeaby', 'buta', 'boy', '...",3,contentment,hands keep bayeeaby buta boy jewelry get mine ...
2296,And I Love Her,The Beatles,give love thats saw love youd love love gives ...,"['love', 'die', 'brings', 'shine', 'bright', '...",3,contentment,love die brings shine bright dark stars mine s...
3236,Rock 'n' Roll Music,The Beatles,rock roll music let hear rock roll music old w...,"['rock', 'roll', 'music', 'againt', 'dance', '...",3,contentment,rock roll music againt dance jazz sounded darn...
4689,The Long and Winding Road,The Beatles,long winding road leads door never disappear i...,"['winding', 'road', 'long', 'leads', 'many', '...",5,sadness,winding road long leads many ive lead stan tim...
5430,You Can't Do That,The Beatles,got something say might cause pain catch talki...,"['flat', 'let', 'leave', 'talking', 'told', 'g...",5,sadness,flat let leave talking told gon think theyd na...
6551,PS I Love You,The Beatles,write letter send love remember ill always lov...,"['ps', 'love', 'letter', 'write', 'send', 'tre...",3,contentment,ps love letter write send treasure till foreve...
6718,Blackbird/Yesterday,The Beatles,galveston oh galveston still hear seawinds blo...,"['galveston', 'still', 'seawinds', 'seawaves',...",2,fear,galveston still seawinds seawaves standing cra...
6791,Help! [Live][Version],The Beatles,help need somebody help anybody help know need...,"['help', 'younger', 'needed', 'selfassured', '...",3,contentment,help younger needed selfassured anybodys days ...


##### Select the song index:

In [91]:
song_index = int(input("Enter the song index: "))

Enter the song index:  4689


In [92]:
song = dataframe_songs.loc[song_index]
print("You have chosen: {}".format(song['track']), "by {}".format(song['artist']))

You have chosen: The Long and Winding Road by The Beatles


In [93]:
song_emotion = dataframe_songs.loc[song_index]['label']
song_emotion

'sadness'

In [94]:
emotion_mapping = {
    'contentment': 0,
    'sadness': 1,
    'excitement': 2,
    'fear': 3,
    'anger': 4,
    'something else': 5
}

song_emotion_index = emotion_mapping[song_emotion]
song_emotion_index

1

In [95]:
# Find the correct index on the dataframe

emotion_dfs = {}
for emotion in dataframe_songs['label'].unique():
    emotion_dfs[emotion] = dataframe_songs[dataframe_songs['label'] == emotion].copy()

emotion_df = emotion_dfs[song_emotion]
matching_rows = emotion_df[(emotion_df['track'] == song['track']) & (emotion_df['artist'] == song['artist'])].index

for row in matching_rows:
    print(row)

i = 0
for index, row in emotion_df.iterrows():
    if row['track'] == song['track'] and row['artist'] == song['artist']:
        print("Row number:", index, i)
        index = i
        break
    i+=1

4689
Row number: 4689 1818


In [96]:
df_artemis = pd.read_csv('data/artemis_dataset_release_v0.csv')

In [97]:
emotion_dfs_artemis = {}
for emotion in df_artemis['emotion'].unique():
    emotion_dfs_artemis[emotion] = df_artemis[df_artemis['emotion'] == emotion].copy()

In [98]:
print(embeddings_lyrics[0].shape)
print(embeddings_lyrics[1].shape)
print(embeddings_lyrics[2].shape)
print(embeddings_lyrics[3].shape)
print(embeddings_lyrics[4].shape)
print(embeddings_lyrics[5].shape)

# embeddings_lyrics[1][4689]

torch.Size([3568, 384])
torch.Size([5789, 384])
torch.Size([1623, 384])
torch.Size([1775, 384])
torch.Size([1798, 384])
torch.Size([432, 384])


In [99]:
print(index)
if song_emotion_index == 0:
    similarity = torch.matmul(embeddings_lyrics[0][index], embeddings_artemis[2].T)
elif song_emotion_index == 1:
    similarity = torch.matmul(embeddings_lyrics[1][index], embeddings_artemis[1].T)
elif song_emotion_index == 2:
    similarity = torch.matmul(embeddings_lyrics[2][index], embeddings_artemis[5].T)
elif song_emotion_index == 3:
    similarity = torch.matmul(embeddings_lyrics[3][index], embeddings_artemis[6].T)
elif song_emotion_index == 4:
    similarity = torch.matmul(embeddings_lyrics[4][index], embeddings_artemis[8].T)
elif song_emotion_index == 5:
    similarity = torch.matmul(embeddings_lyrics[5][index], embeddings_artemis[0].T)

print(similarity)
print(similarity.shape)

1818
tensor([0.0796, 0.0888, 0.2586,  ..., 0.1082, 0.0851, 0.0701])
torch.Size([49061])


In [100]:

top_indices = np.argsort(similarity)[-1500:]
#print(top_indices.numpy().tolist())

print(emotion_dfs[song_emotion].iloc[index]['top_words'])
most_similar_sentences = emotion_dfs_artemis[song_emotion].iloc[top_indices.numpy().tolist()]
most_similar_sentences

# emotion_dfs_artemis[song_emotion].iloc[26833]

['winding', 'road', 'long', 'leads', 'many', 'ive', 'lead', 'stan', 'times', 'door']


,art_style,painting,emotion,utterance,repetition
257594,Art_Nouveau_Modern,nicholas-roerich_the-oldest-the-wisest-1944,sadness,The man seems to be looking as if he has lost ...,10
185024,Abstract_Expressionism,jackson-pollock_mural,sadness,A tangled life often leads to a web of confusion.,5
202347,Minimalism,gene-davis_peach-glow-1958,sadness,"the emptiness of it all, just lines and shadows",6
169225,Contemporary_Realism,lucian-freud_cyclamen-1964,sadness,Fading dying flowers.,5
149363,Post_Impressionism,henri-le-fauconnier_vielle-femme,sadness,Grandma hopes the Uber arrives before she turn...,5
...,...,...,...,...,...
224024,Pointillism,giacomo-balla_bankruptcy-1902,sadness,This door feels lonely and left behind,5
251336,Post_Impressionism,maxime-maufra_morning-in-winter-1905,sadness,This long and winding road looks very cold and...,5
294128,Realism,ivan-shishkin_the-road,sadness,Seeing this winding road lead to an abandoned-...,6
96629,Abstract_Expressionism,brice-marden_vine-1993,sadness,The lines are long but don't lead anywhere exc...,6


## Create dataset for song

In [101]:
import os
import shutil

In [103]:
path_destination = 'pickSongGenerateDS/no_label/'
for index, row in most_similar_sentences.iterrows():
    directory = 'data/' + row['art_style']
    file = row['painting'] + '.jpg'
    
    for root, dirs, files in os.walk(directory):
        if file in files:
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(path_destination, file)
            shutil.copyfile(source_file_path, destination_file_path)
            #print("File copied successfully!")
    
    

#### The generated images dataset is located in pickSongGeneratedDS/